In [1]:
# %matplotlib
# !git reset master --hard && git pull

In [2]:
# !wget http://www.openslr.org/resources/12/dev-clean.tar.gz
# !tar -zxf dev-clean.tar.gz

In [3]:
# !wget https://raw.githubusercontent.com/pyannote/pyannote-audio/develop/tutorials/data_preparation/download_ami.sh
# !mkdir ami
# !bash download_ami.sh ami

In [4]:
# !wget https://raw.githubusercontent.com/pyannote/pyannote-audio/develop/tutorials/data_preparation/AMI/MixHeadset.development.rttm
# !wget https://raw.githubusercontent.com/pyannote/pyannote-audio/develop/tutorials/data_preparation/AMI/MixHeadset.test.rttm
# !wget https://raw.githubusercontent.com/pyannote/pyannote-audio/develop/tutorials/data_preparation/AMI/MixHeadset.train.rttm

In [5]:
import malaya_speech.train as train
import numpy as np
import malaya_speech
from tqdm import tqdm
import random

np.seterr(divide='raise', invalid='raise')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [6]:
import librosa

def random_stretch(samples, low = 0.5, high = 1.3):
    input_length = len(samples)
    stretching = samples.copy()
    random_stretch = np.random.uniform(low = low, high = high)
    stretching = librosa.effects.time_stretch(
        stretching.astype('float'), random_stretch
    )
    return stretching

def random_pitch(samples, low = 0.5, high = 1.0):
    y_pitch_speed = samples.copy()
    length_change = np.random.uniform(low = low, high = high)
    speed_fac = 1.0 / length_change
    tmp = np.interp(
        np.arange(0, len(y_pitch_speed), speed_fac),
        np.arange(0, len(y_pitch_speed)),
        y_pitch_speed,
    )
    minlen = min(y_pitch_speed.shape[0], tmp.shape[0])
    y_pitch_speed *= 0
    y_pitch_speed[:minlen] = tmp[:minlen]
    return y_pitch_speed

In [7]:
import os
from glob import glob

ami = glob('ami/amicorpus/*/*/*.wav')
ami = {os.path.split(f)[1].replace('.wav', ''): f for f in ami}
rttm = glob('*.rttm')
len(ami), len(rttm)

(171, 3)

In [8]:
import pandas as pd
import random
from tqdm import tqdm

maxlen = 0.3
selected_frames = [30, 90]
functions = [random_pitch, random_stretch]

In [9]:
files = glob('../speech-bahasa/LibriSpeech/test-clean/*/*/*.flac')
files.extend(glob('LibriSpeech/dev-clean/*/*/*.flac'))
len(files)

5323

In [10]:
noises = glob('../noise/noise/*.wav')
noises = [malaya_speech.astype.int_to_float(malaya_speech.utils.read.wav(n)[0]) for n in tqdm(noises)]
len(noises)

100%|██████████| 182/182 [00:39<00:00,  4.57it/s]


182

In [12]:
def generator(sr = 16000):
    
    for file in tqdm(files):
        y, sr = malaya_speech.utils.read.flac(file)
        vad = malaya_speech.vad.webrtc(minimum_amplitude = int(np.quantile(np.abs(y), 0.3)))
        frames = malaya_speech.utils.generator.frames(y, 30, sr, False)
        frames = [(frame, vad(frame)) for frame in frames]
        grouped = malaya_speech.vad.group_vad(frames)

        x, y = [], []
        for frame, label in grouped:
            if frame.duration < maxlen:
                x.append(malaya_speech.astype.int_to_float(frame.array))
                y.append(label)
            for s in selected_frames:
                try:
                    frames = malaya_speech.utils.generator.frames(frame.array, s, sr, False)
                    x.extend([malaya_speech.astype.int_to_float(f.array) for f in frames])
                    y.extend([label] * len(frames))
                except:
                    pass

        for i in range(len(x)):
            for _ in range(random.randint(0, 3)):
                if y[i]:
                    factor = random.uniform(0.1, 0.5)
                else:
                    factor = random.uniform(0.4, 0.9)

                n = random.choice(noises)
                added_noise = malaya_speech.augmentation.waveform.add_noise(x[i],n, 
                                                                            factor = factor)
                x.append(added_noise)
                y.append(y[i])
                
            for _ in range(random.randint(0, 3)):
                for f in functions:
                    x.append(f(x[i]))
                    y.append(y[i])

        for i in range(len(x)):

            yield {
                'waveforms': x[i].tolist(),
                'targets': [int(y[i])],
            }
        
generator = generator()

In [13]:
import os
import tensorflow as tf

os.system('rm vad2/data/*')
DATA_DIR = os.path.expanduser('vad2/data')
tf.gfile.MakeDirs(DATA_DIR)

In [14]:
shards = [{'split': 'train', 'shards': 1000}, {'split': 'dev', 'shards': 1}]
train.prepare_dataset(generator, DATA_DIR, shards, prefix = 'vad')

  0%|          | 0/5323 [00:00<?, ?it/s]


INFO:tensorflow:Generating case 0.


  1%|          | 52/5323 [01:15<2:11:24,  1.50s/it]

INFO:tensorflow:Generating case 100000.


  2%|▏         | 97/5323 [02:36<2:44:43,  1.89s/it]

...

100%|█████████▉| 5316/5323 [1:56:26<00:10,  1.43s/it]

INFO:tensorflow:Generating case 8800000.


100%|██████████| 5323/5323 [1:56:37<00:00,  1.31s/it]


INFO:tensorflow:Generated 8810401 Examples
INFO:tensorflow:Shuffling data...
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`



INFO:tensorflow:Data shuffled.
